In [ ]:
!pip install tensorflow

In [3]:
import tensorflow as tf
print(tf.__version__)

2.18.0


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
import zipfile

# Unzip it
with zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/archive (2).zip', 'r') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/Colab Notebooks/churn_data')

# Load the CSV (assuming name is Churn_Modelling.csv inside the zip)
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/churn_data/Churn_Modelling.csv')
df.head()


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
X=df.iloc[:,3:13]
y=df.iloc[:,13]

In [8]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [9]:
y

,Exited
0,1
1,0
2,1
3,0
4,0
...,...
9995,0
9996,0
9997,1
9998,1


In [10]:
geography=pd.get_dummies(X['Geography'], drop_first=True, dtype=int)
gender=pd.get_dummies(X['Gender'], drop_first=True, dtype=int)

In [11]:
X=X.drop(['Geography','Gender'], axis=1)

In [12]:
X=pd.concat([X,geography,gender], axis=1)

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [14]:
#Feature Scaling
#Required in algos having 1) distance 2) gradient descent
#Linear regression, Logistic Regression, KNN, K Means

In [15]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [16]:
#What is fit() doing?

'''
It calculates:
Mean of each feature in X_train
Standard deviation (std) of each feature in X_train
'''

#sc.transform(X_train)

'''
It applies this formula to every value:
scaled_value = (original_value - mean) / std
'''


'\nIt applies this formula to every value:\nscaled_value = (original_value - mean) / std\n'

In [17]:
X_train

array([[ 0.35649971, -0.6557859 ,  0.34567966, ..., -0.57946723,
        -0.57638802,  0.91324755],
       [-0.20389777,  0.29493847, -0.3483691 , ...,  1.72572313,
        -0.57638802,  0.91324755],
       [-0.96147213, -1.41636539, -0.69539349, ..., -0.57946723,
         1.73494238,  0.91324755],
       ...,
       [ 0.86500853, -0.08535128, -1.38944225, ..., -0.57946723,
        -0.57638802, -1.09499335],
       [ 0.15932282,  0.3900109 ,  1.03972843, ..., -0.57946723,
        -0.57638802,  0.91324755],
       [ 0.47065475,  1.15059039, -1.38944225, ...,  1.72572313,
        -0.57638802,  0.91324755]])

In [18]:
X_train.shape

(8000, 11)

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LeakyReLU, PReLU, ELU, ReLU
from tensorflow.keras.layers import Dropout

In [20]:
#Sequential means we'll be able to do forward and backward propagation
#Dense -- to add layers (input, hidden, output)
#activation function -- ReLU and all
#Dropout -- is like a regularization parameter, prevents overfitting, if dropout ratio is say 0.3 for a layer then 30% of neurons in that layer will get deactivated

In [21]:
classifier=Sequential()

In [44]:
#Input Layer
classifier.add(Dense(units=11, activation='relu'))  #Shape of X is (8000,11) we need 11 inputs in out classifier

In [45]:
#First hidden layer
classifier.add(Dense(units=6, activation='relu'))
#classifier.add(Dropout(0.2))

In [46]:
#Second hidden layer
classifier.add(Dense(units=6,activation='relu'))

In [47]:
#Adding the output layer
classifier.add(Dense(units=1,activation='sigmoid'))

In [48]:
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])   #adam is the best optimizer in practice and uses a learning rate of 0.01 by default

In [49]:
classifier.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8045 - loss: 0.5807 - val_accuracy: 0.8035 - val_loss: 0.4964
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7941 - loss: 0.5067 - val_accuracy: 0.8035 - val_loss: 0.4597
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7916 - loss: 0.4655 - val_accuracy: 0.8035 - val_loss: 0.4301
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7952 - loss: 0.4370 - val_accuracy: 0.8285 - val_loss: 0.4185
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8306 - loss: 0.4197 - val_accuracy: 0.8545 - val_loss: 0.3724
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8561 - loss: 0.3666 - val_accuracy: 0.8540 - val_loss: 0.3562
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8627 - loss: 0.3478 - val_accuracy: 0.8555 - val_loss: 0.3501
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8594 - loss: 0.3509 - val_accuracy: 0.

In [50]:
#What if you want to give your own learning rate

'''
import tensorflow
tensorflow.keras.optimizers.Adam(learning_rate=0.01)
'''

'\nimport tensorflow\ntensorflow.keras.optimizers.Adam(learning_rate=0.01)\n'

In [51]:
#Predicting on Test set results
y_pred=classifier.predict(X_test)
y_pred=(y_pred>0.5)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [52]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, y_pred)
cm

array([[1534,   73],
       [ 209,  184]])

In [53]:
from sklearn.metrics import accuracy_score
score=accuracy_score(y_test, y_pred)
score

0.859

In [54]:
#Get the updated weights
classifier.get_weights()

[array([[-0.06024382,  0.27657193,  0.0589854 , -0.14135781, -0.03075529,
          0.41568252, -0.10485846, -0.13042116,  0.23920122,  0.07090107,
          0.05102147],
        [ 1.1817774 ,  0.16760378, -0.1117954 ,  0.13372827, -0.60074866,
          0.59201336, -0.32375395, -0.2991082 , -0.31092876,  0.32932872,
         -0.42892253],
        [ 0.01406203, -0.05383319,  0.29993138, -0.10102244,  0.14646633,
         -0.11748424, -0.6100904 ,  0.20586964, -0.04866455,  0.07340938,
          0.01755569],
        [-0.22897637,  0.67724895,  0.15690212,  0.16994378, -0.47117126,
         -0.00838694, -0.3318565 ,  0.2141381 , -1.0215964 , -0.0462098 ,
         -0.2888391 ],
        [-0.7252291 , -0.24847238, -0.50741327,  0.44321308, -0.561621  ,
         -0.20811814,  0.14258991,  0.00700739, -0.05925886,  1.0978827 ,
          0.07547868],
        [ 0.06885912,  0.01616753,  0.20351323, -0.04968314, -0.0200043 ,
          0.02177819,  0.21318828, -0.03912257, -0.06298961, -0.0590102